In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [31]:
movies = pd.read_csv('./ml-1m/ml-1m/movies.dat', sep='::',
                     header=None, engine='python', encoding='latin-1')
users = pd.read_csv('./ml-1m/ml-1m/users.dat', sep='::', header=None,
                    engine='python', encoding='latin-1')
ratings = pd.read_csv('./ml-1m/ml-1m/ratings.dat', sep='::',
                      header=None, engine='python', encoding='latin-1')

In [32]:
training_set = pd.read_csv('ml-100k/ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [33]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [34]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data


training_set = convert(training_set)
test_set = convert(test_set)

In [35]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

the input is nn.Module. It is the inherited parent class.
it is a class in pytorch that creates and activates the neural network.
super function is used to get methods from nn.Module

fc# is the full connection number. 
number of layers (incoming, outgoing)
Linear is the layer type



In [36]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 97)
        self.fc2 = nn.Linear(97, 37)
        self.fc3 = nn.Linear(37, 83)
        self.fc4 = nn.Linear(83, 59)
        self.fc5 = nn.Linear(59, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        x = self.fc5(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.Adamax(sae.parameters(), lr=0.01, weight_decay=0.5)

In [37]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / \
                float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1
            optimizer.step()
    print(f'epoch: {epoch} loss: {train_loss/s}')

/home/guess/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


epoch: 1 loss: 1.4194051027297974
epoch: 2 loss: 1.0619930028915405
epoch: 3 loss: 1.0541507005691528
epoch: 4 loss: 1.0507789850234985
epoch: 5 loss: 1.0482618808746338
epoch: 6 loss: 1.0462329387664795
epoch: 7 loss: 1.04392409324646
epoch: 8 loss: 1.041454553604126
epoch: 9 loss: 1.0387554168701172
epoch: 10 loss: 1.0369409322738647
epoch: 11 loss: 1.0357064008712769
epoch: 12 loss: 1.0346179008483887
epoch: 13 loss: 1.0339629650115967
epoch: 14 loss: 1.0333806276321411
epoch: 15 loss: 1.032731056213379
epoch: 16 loss: 1.0321931838989258
epoch: 17 loss: 1.031589150428772
epoch: 18 loss: 1.0309561491012573
epoch: 19 loss: 1.030419945716858
epoch: 20 loss: 1.0300483703613281
epoch: 21 loss: 1.0317261219024658
epoch: 22 loss: 1.0315182209014893
epoch: 23 loss: 1.0301690101623535
epoch: 24 loss: 1.0295594930648804
epoch: 25 loss: 1.0290921926498413
epoch: 26 loss: 1.0282353162765503
epoch: 27 loss: 1.0272399187088013
epoch: 28 loss: 1.02608060836792
epoch: 29 loss: 1.024830937385559
epo

KeyboardInterrupt: 

In [ ]:
test_loss = 0
s = 0
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1
print(f'test loss: {test_loss/s}')

- AdaGrad: test loss = 0.9383 | final loss = 0.8790
- AdaDelta: Garbage!!!
- ASGD: test loss = 0.9676 | final loss = 0.9020
- Adam: test loss = 0.9307 | final loss = 0.8667
- AdaMax: test loss = 0. | final loss = 0.
- RMSProp: test loss = 0.9722 | final loss = 0.9222 
- RProp: test loss =  | final loss = 
- SGD: test loss =  | final loss = 
- ASGD: test loss =  | final loss = 